# Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import copy
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder

import tqdm
import matplotlib.pyplot as plt
import time
import os
import copy


# Setting device for the computation

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Resizing all the images to the same size

In [51]:
train_transform = transforms.Compose([
        #TODO:think a better transformation pipeline
        #naive transformation
        transforms.Resize((64,64)),
        transforms.ToTensor()
    ])
test_transform = transforms.Compose([
        #TODO:think a better transformation pipeline
        #naive transformation
        transforms.Resize((64,64)),
        transforms.ToTensor()
    ])

# Loading the train dataset

In [62]:
train_dir='./dataset/GTSRB/train'

train_dataset = datasets.ImageFolder(train_dir,train_transform)

train_loader = DataLoader(train_dataset, batch_size=200,shuffle=True, num_workers=4)

train_total_steps = len(train_loader)
train_size = len(train_dataset)
class_names = train_dataset.classes

print('Total steps for training:',train_total_steps)
print('Train size:',train_size)
print('Class names:',class_names)

Total steps for training: 393
Train size: 39209
Class names: ['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009', '00010', '00011', '00012', '00013', '00014', '00015', '00016', '00017', '00018', '00019', '00020', '00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028', '00029', '00030', '00031', '00032', '00033', '00034', '00035', '00036', '00037', '00038', '00039', '00040', '00041', '00042']


# Loading the test dataset

In [63]:
test_dir='./dataset/GTSRB/test'

test_dataset = datasets.ImageFolder(test_dir,test_transform)

test_loader = DataLoader(test_dataset,batch_size=200, shuffle=True, num_workers=0)

test_size = len(test_dataset)
class_names = test_dataset.classes

print('Test size:',train_size)
print('Class names:',class_names)

Test size: 39209
Class names: ['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009', '00010', '00011', '00012', '00013', '00014', '00015', '00016', '00017', '00018', '00019', '00020', '00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028', '00029', '00030', '00031', '00032', '00033', '00034', '00035', '00036', '00037', '00038', '00039', '00040', '00041', '00042']


# Defining the training phase

In [68]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    #best_model_wts = copy.deepcopy(model.state_dict())
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0

        #iterates over the batches
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
    
            # forward pass
            outputs = model(images) #compute predictions
            _, preds = torch.max(outputs, 1) #maximum value along dimension 1
            loss = criterion(outputs, labels)

            # backward + optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #prints the stats every 20 steps (20 batches performed)
            if (i+1) % 20 == 0:
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{train_total_steps}], Loss: {loss.item():.4f}')
    
            # statistics
            running_loss += loss.item() * images.size(0) # scalar loss * n.of samples in the batch: necessary to average the loss over the batch
            running_corrects += torch.sum(preds == labels.data)
            
            #to update weights
            scheduler.step()
    
        epoch_loss = running_loss / train_size
        epoch_acc = running_corrects.double() / train_size
    
        print('{} Epoch {} Loss: {:.4f} Acc: {:.4f}'.format(
            'Train phase - ',epoch, epoch_loss, epoch_acc))
    
    print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    return model

# Defining the model using ResNet18 as backbone

In [69]:
from torchvision.models import ResNet18_Weights

#### ConvNet as fixed feature extractor ####
# freeze all the network parameters except the final layer settings, to not compute the gradients backward
model_conv = torchvision.models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 43)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized
optimizer_conv = torch.optim.Adam(model_conv.fc.parameters(), lr=0.01)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv,
                        exp_lr_scheduler, num_epochs=20)

Epoch 0/19
----------
Epoch [1/20], Step [20/393], Loss: 3.0241
Epoch [1/20], Step [40/393], Loss: 3.0280
Epoch [1/20], Step [60/393], Loss: 2.9532
Epoch [1/20], Step [80/393], Loss: 2.7694
Epoch [1/20], Step [100/393], Loss: 2.9545
Epoch [1/20], Step [120/393], Loss: 3.3594
Epoch [1/20], Step [140/393], Loss: 2.8523
Epoch [1/20], Step [160/393], Loss: 3.0209
Epoch [1/20], Step [180/393], Loss: 2.4268
Epoch [1/20], Step [200/393], Loss: 3.0993
Epoch [1/20], Step [220/393], Loss: 2.6250
Epoch [1/20], Step [240/393], Loss: 2.2967
Epoch [1/20], Step [260/393], Loss: 2.7128
Epoch [1/20], Step [280/393], Loss: 2.8641
Epoch [1/20], Step [300/393], Loss: 3.0498
Epoch [1/20], Step [320/393], Loss: 3.0260
Epoch [1/20], Step [340/393], Loss: 3.2138
Epoch [1/20], Step [360/393], Loss: 2.9020
Epoch [1/20], Step [380/393], Loss: 3.4047
train Epoch 0 Loss: 2.9393 Acc: 0.3288
Epoch 1/19
----------
Epoch [2/20], Step [20/393], Loss: 3.2393
Epoch [2/20], Step [40/393], Loss: 3.0915
Epoch [2/20], Step [

KeyboardInterrupt: 

# Saving the trained model

In [172]:
print('Finished Training')
os.makedirs('./models', exist_ok=True)
PATH = './models/trained_model.pth'
torch.save(model_conv, PATH)

Finished Training


# Loading the model

In [5]:
trained_model=torch.load('./models/trained_model.pth',map_location=device)

# Evaluating the model

In [15]:
def test_model(model, dataloader, dataset_size):
    model.eval()  # Set model to evaluation mode
    
    running_corrects = 0

    with torch.no_grad():  # Disable gradient calculation
        for i, (images, labels) in enumerate(dataloader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
    
            # Statistics
            running_corrects += torch.sum(preds == labels.data)
            if (i+1) % 50 == 0:
                print ( f'Evaluating: [{i+1}/{test_size}]')

    # Calculate accuracy
    test_acc = running_corrects.double() / dataset_size

    print('Test Acc: {:.4f}'.format( test_acc))
    

In [16]:
test_model(trained_model, test_loader, test_size)

Evaluating: [50/12630]
Test Acc: 0.3157
